# 0. Instalar Dependências do atari

In [ ]:
!pip install atari-py

In [ ]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

In [ ]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install tensorflow==2.3.1 gym keras-rl2 

In [ ]:
# Download de jogos de atari (use envs.registry.all() para ver todos os roms)
!wget http://www.atarimania.com/roms/Roms.rar
!unrar x Roms.rar
!python -m atari_py.import_roms /content
# Caso rode esta célula já tendo os roms baixados, lembrar de prencher a barra que aparecerá no código para pular o download de arquivos repetidos

In [ ]:
# Remova " > /dev/null 2>&1" se quiser ver mais detalhes
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
# Esta etapa evita alguns possíveis erros
%%bash

# Instala mais algumas dependências
apt-get install -y xvfb x11-utils

pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

apt-get install swig

Obs: É possível que o ambiente de execução do colab apresente erros e forçe sua reinicialização, neste caso, não é necessário rodar todos estes códigos acima novamente, pois já podem estar instalados.

# 1. Importar biblioteca e preparar display para visualização os jogos

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [ ]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# 2. Teste com OpenAI Gym com ações aleatórias

In [ ]:
# Preparo do ambiente
# Escolha do jogo aqui
env = wrap_env(gym.make('SpaceInvaders-v0').env)
# Captura dimensões da observação; compatível apenas com observações de imagem.
height, width, channels = env.observation_space.shape
# Armazena quantidade de ações
actions = env.action_space.n

In [ ]:
# Descreve as ações disponíveis
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
# Demonstração do ambiente com ações aleatórias, esta etapa pode ser pulada, pois é a penas uma visualização

episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5]) # escolhe aleatoriamente entre as ações disponíveis
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

# 2. Criação do modelo de Deep Learning através da biblioteca Keras

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
# Este método vai escanear a tela do jogo em chuncks determinados e filtrar essa informação para a criação do modelo

def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
# Na ocorrência do erro: 
#"Tensor("dense_3/kernel/Read/ReadVariableOp:0", shape=(256, 7), dtype=float32) must be from the same graph as Tensor("dense_1/Relu:0", shape=(None, 256), dtype=float32) 
#(graphs are <tensorflow.python.framework.ops.Graph object at 0x7fbf42f9db10> and FuncGraph(name=keras_graph, id=140459438308496))."
# basta deletar o modelo e criá-lo novamente.

del model

In [ ]:
model = build_model(height, width, channels, actions)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 6)                

# 3. Construção do agente com o Keras

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000  
                  ) 
    return dqn

In [ ]:
## Na ocorrência do erro: 
#"Tensor("dense_3/kernel/Read/ReadVariableOp:0", shape=(256, 7), dtype=float32) must be from the same graph as Tensor("dense_1/Relu:0", shape=(None, 256), dtype=float32) 
#(graphs are <tensorflow.python.framework.ops.Graph object at 0x7fbf42f9db10> and FuncGraph(name=keras_graph, id=140459438308496))."
# basta deletar o modelo e criá-lo novamente.

dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

In [ ]:
# Rodar apenas se estiver treinando um modelo, pode pular esta etapa se estiver carregando pesos pré-treinados
# verbose = 1 mostrará uma barra de carregamento, verbose = 2 mostra resultado a cada conjunto de episódios
# nb_steps = 10_000_000 é recomendado para jogos de atari que superam jogadores humanos

dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

# 4. Testa o modelo e mostra o vídeo

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Episode 1: reward: 170.000, steps: 667
Episode 2: reward: 140.000, steps: 666
Episode 3: reward: 300.000, steps: 1175
Episode 4: reward: 155.000, steps: 658
Episode 5: reward: 160.000, steps: 839
Episode 6: reward: 195.000, steps: 999
Episode 7: reward: 130.000, steps: 698
Episode 8: reward: 215.000, steps: 760
Episode 9: reward: 170.000, steps: 647
Episode 10: reward: 150.000, steps: 636
178.5


In [ ]:
show_video()

# 5. Salvar e Carregar Agente

In [ ]:
dqn.save_weights('SavedWeights/teste/dqn_weights.h5f')

In [ ]:
del model

In [ ]:
dqn.load_weights('/content/drive/MyDrive/1m/dqn_weights.h5f')

# 6. Referências

Deep Reinforcement Learning for Atari Games Python Tutorial | AI Plays Space Invaders. Realização de Nicholas Renotte. [S.I.]: [S.I], 2020. (38 min.), son., color. Disponível em: https://www.youtube.com/watch?v=hCeJeq8U0lo&ab_channel=NicholasRenotte. Acesso em: 26 jul. 2021.